<a href="https://colab.research.google.com/github/elhamadjidi/Natural-Language-Processing-with-Disaster-Tweets/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torchvision

In [2]:
!pip install ydata-profiling

  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.25.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


**Reading files**

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/drive/MyDrive/Colab Notebooks/kaggle/nlp-getting-started/train.csv
/content/drive/MyDrive/Colab Notebooks/kaggle/nlp-getting-started/test.csv
/content/drive/MyDrive/Colab Notebooks/kaggle/nlp-getting-started/sample_submission.csv


In [14]:
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/nlp-getting-started/test.csv")
test_data.head()
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/nlp-getting-started/train.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


**Now Runtime -> Restart session and then Run the below**

# Data visulalization

In [ ]:
from ydata_profiling import ProfileReport
train_profile = ProfileReport(train_data, title="train")
train_profile.to_notebook_iframe()

In [5]:
train_data.groupby('keyword')['id'].count()

keyword
ablaze                 36
accident               35
aftershock             34
airplane%20accident    35
ambulance              38
                       ..
wounded                37
wounds                 33
wreck                  37
wreckage               39
wrecked                39
Name: id, Length: 221, dtype: int64

In [6]:
keyword_count = train_data.groupby('keyword')['id'].count() #This groups the DataFrame train by the 'keyword' column and then counts the occurrences of 'id' within each group.
keyword_count_dic = pd.DataFrame({'keyword':keyword_count.index,'count':keyword_count.values}).sort_values(by=['count'])
top = keyword_count_dic.tail(20) #This sorts the DataFrame top by the 'count' column in ascending order and then selects the last 20 rows
bottom = keyword_count_dic.head(20)

# Location column need some work:


1.   Extract location from the text and fill the blanks with the location it found
2.   To standardize location names in your DataFrame column and use a consistent representation such as "NYC" instead of various variations like "NY", "NYC", "ny", etc



Here we try to extract location from the text in each row and when location column is empty use this location to fill it.

In [15]:
import pandas as pd
import spacy

# Load spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Function to extract location names from text
def extract_locations(text):
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    return locations

# Iterate over rows
for index, row in train_data.iterrows():
    if pd.isnull(row['location']):  # Check if 'location' is null
        text = row['text']
        if pd.notnull(text):  # Check if 'text' is not null
            # Extract location names from text
            locations = extract_locations(text)
            if locations:
                train_data.at[index, 'location'] = locations[0]  # Update 'location' with the first extracted location
# Iterate over rows
for index, row in test_data.iterrows():
    if pd.isnull(row['location']):  # Check if 'location' is null
        text = row['text']
        if pd.notnull(text):  # Check if 'text' is not null
            # Extract location names from text
            locations = extract_locations(text)
            if locations:
                test_data.at[index, 'location'] = locations[0]  # Update 'location' with the first extracted location

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


**To standardize location names in your DataFrame column and use a consistent representation such as "NYC" instead of various variations like "NY", "NYC", "ny", etc**

In [16]:
import re
# checking the location and how many of tweets in that location
location_counts = train_data['location'].value_counts().to_dict()
#Since there are so many similar names for New York i want to change all of those to NYC
# Define the pattern you want to replace
pattern = r'\b(?:new york(?: city)?|nyc(?:,? ny| city| brooklyn|, usa)?|ny(?:,? ny)?|brooklyn, nyc|nyc|bronx, nyc)\b'

# Replace the values using regex
train_data['location'] = train_data['location'].str.replace(pattern, 'nyc', regex=True)
test_data['location'] = test_data['location'].str.replace(pattern, 'nyc', regex=True)

location_counts = train_data['location'].value_counts().to_dict()

Some experiment seeing how location can be related to how truthful the tweet is.

In [17]:
# for each location make a table of what is the rate of corrects with them
from_pho = train_data.loc[train_data.location == 'texas']["target"]
rate_from_pho = sum(from_pho)/len(from_pho)

print("% of twitters from texas that were correct:", rate_from_pho)

% of twitters from texas that were correct: 0.0


See the number of unique locations and how many of tweets are truthful in each location. basically the extensive version of the code above.

In [18]:
# Get unique locations
unique_locations = train_data['location'].unique()

# Create a list to store results
results = []

# Calculate rate for each location and append to results list
for location in unique_locations:
    if pd.notna(location):  # Skip NaN values
        from_location = train_data.loc[train_data['location'] == location, "target"]
        rate_from_location = from_location.mean() * 100
        results.append((location, rate_from_location, len(from_location)))

# Sort the results list by rate and length of from_location
sorted_results = sorted(results, key=lambda x: (x[1], x[2]), reverse=True)

# # Print the sorted results
# for result in sorted_results:
#     print(f"% of twitters from {result[0]} that were correct: {result[1]:.2f}%, Count: {result[2]}")

In [19]:
len(unique_locations)

3518

# Keyword column has one simple work:

1.   For columns that are blank in keyword column looks to find 'fire' or 'forest' and write fire on the keyword column. it does the same thing for 'flood' and similar words to it.




In [20]:
# Define regex pattern to search for words related to "fire", "forest", and "flood"
fire_forest_pattern = r'\b(fire|forest)\b'
flood_pattern = r'\b(flood|#flood|#flooding)\b'

# Function to update 'keyword' column based on text content
def update_keyword(row):
    if pd.isnull(row['keyword']) or row['keyword'].strip() == '':
        if re.search(fire_forest_pattern, row['text'], flags=re.IGNORECASE):
            return 'fire'
        elif re.search(flood_pattern, row['text'], flags=re.IGNORECASE):
            return 'flood'
    return row['keyword']

# Apply the function to update the 'keyword' column
train_data['keyword'] = train_data.apply(update_keyword, axis=1)

# General data cleaning:
Some changes in the text, location and keyword columns.
changes include:


1.   text column value should be string
2.   all columns should be lower case
3.   Remove trailing whitespace from each column
4.   Remove special characters and numbers
5.   Lemmatize in keyword column
6.   Remove URL from the text
7.   Remove %20 from keyword



In [21]:
import re

def preprocess_data(data):
    # Convert 'text' column to string type
    data['text'] = data['text'].astype(str)
    # Convert 'location' column to string type (if needed)
    # data['location'] = data['location'].astype(str)

    # Convert 'text', 'location', and 'keyword' columns to lowercase
    data[['location', 'text', 'keyword']] = data[['location', 'text', 'keyword']].apply(lambda x: x.str.lower())

    # Remove trailing whitespace from 'location' and 'text' columns
    data[['location', 'text']] = data[['location', 'text']].apply(lambda x: x.str.rstrip())

    # Remove special characters and numbers from 'text' column
    data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

    # Remove URLs from 'text' column
    data['text'] = data['text'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+|http://t\.co/\S+', '', x))

    # Remove '%20' from 'keyword' column
    data['keyword'] = data['keyword'].str.replace('%20', '')
    # Apply lambda function to 'location' column to remove special characters and numbers
    data['location'] = data['location'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x) if isinstance(x, str) else x)

    # # Replace missing values

    data['keyword'] = data['keyword'].fillna('None')
    data['location'] = data['location'].fillna('None')




    return data

# Apply preprocessing function to both training and test datasets
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

In [22]:
test_data.isnull().sum()
train_data.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [23]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess data
# Assume train_data and test_data are already loaded DataFrames

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Encode text data
train_encodings_text = tokenizer(list(train_data['text']), truncation=True, padding=True, return_tensors='pt')
test_encodings_text = tokenizer(list(test_data['text']), truncation=True, padding=True, return_tensors='pt')

# Encode keyword data
train_encodings_keyword = tokenizer(list(train_data['keyword']), truncation=True, padding=True, return_tensors='pt')
test_encodings_keyword = tokenizer(list(test_data['keyword']), truncation=True, padding=True, return_tensors='pt')

# Encode location data
train_encodings_location = tokenizer(list(train_data['location']), truncation=True, padding=True, return_tensors='pt')
test_encodings_location = tokenizer(list(test_data['location']), truncation=True, padding=True, return_tensors='pt')

# Concatenate encodings
train_input_ids = torch.cat((train_encodings_text['input_ids'], train_encodings_keyword['input_ids'], train_encodings_location['input_ids']), dim=1)
train_attention_mask = torch.cat((train_encodings_text['attention_mask'], train_encodings_keyword['attention_mask'], train_encodings_location['attention_mask']), dim=1)

test_input_ids = torch.cat((test_encodings_text['input_ids'], test_encodings_keyword['input_ids'], test_encodings_location['input_ids']), dim=1)
test_attention_mask = torch.cat((test_encodings_text['attention_mask'], test_encodings_keyword['attention_mask'], test_encodings_location['attention_mask']), dim=1)

# Create TensorDatasets
train_val_dataset = TensorDataset(train_input_ids, train_attention_mask, torch.tensor(train_data['target'].values))
# Clear space
del test_data
del train_data

# Split data into training and validation sets
train_dataset, val_dataset = train_test_split(train_val_dataset, test_size=0.2, random_state=42)

test_dataset = TensorDataset(test_input_ids, test_attention_mask)

# Define model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)




model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Define training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training
num_epochs = 25
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
# Validation
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
model.eval()
val_preds = []
val_labels = []
for batch in val_loader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
    logits = outputs.logits
    _, predicted = torch.max(logits, 1)
    val_preds.extend(predicted.cpu().numpy())
    val_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_preds)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.5738673670387393


# other ways

In [ ]:
# import torch
# from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
# from torch.utils.data import DataLoader, TensorDataset
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# from transformers import AdamW, get_linear_schedule_with_warmup

# # Load and preprocess data
# # Assume train_data and test_data are already loaded DataFrames
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# # Function to preprocess and encode multiple columns
# def encoding(columns):
#     encodings = {}
#     for column_name, column_data in columns.items():
#         encodings[column_name] = tokenizer(list(column_data), truncation=True, padding=True, return_tensors='pt')
#     return encodings
# # Preprocess and encode multiple columns for training data
# train_encodings = encoding(train_data)

# # Similarly, you can preprocess and encode columns for test data
# test_encodings = encoding(test_data)

# # Concatenate encoded inputs from different columns
# combined_input_ids = torch.cat((train_encodings['text']['input_ids'],
#                                 train_encodings['keyword']['input_ids'],
#                                 train_encodings['location']['input_ids']), dim=1)
# combined_attention_masks = torch.cat((train_encodings['text']['attention_mask'],
#                                       train_encodings['keyword']['attention_mask'],
#                                       train_encodings['location']['attention_mask']), dim=1)
# # Concatenate encoded inputs from different columns
# test_combined_input_ids = torch.cat((test_encodings['text']['input_ids'],
#                                 test_encodings['keyword']['input_ids'],
#                                 test_encodings['location']['input_ids']), dim=1)
# test_combined_attention_masks = torch.cat((test_encodings['text']['attention_mask'],
#                                       test_encodings['keyword']['attention_mask'],
#                                       test_encodings['location']['attention_mask']), dim=1)
# # Create TensorDatasets
# train_dataset = TensorDataset(combined_input_ids, combined_attention_masks, torch.tensor(train_data['target'].values))
# test_dataset = TensorDataset(test_combined_input_ids, test_combined_attention_masks)

# # Define model
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# # Define training parameters
# optimizer = AdamW(model.parameters(), lr=5e-5)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Training
# num_epochs = 2
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# model.train()
# for epoch in range(num_epochs):
#     print(f'Epoch {epoch + 1}/{num_epochs}')
#     for batch in train_loader:
#         input_ids, attention_mask, labels = batch
#         optimizer.zero_grad()
#         outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
#         loss = outputs.loss #the loss value computed by the model for the current batch of input data.
#         loss.backward() #backpropagation #computes the gradients of the loss with respect to all the model parameters. These gradients represent the direction and magnitude of the change needed in each parameter to decrease the loss.
#         optimizer.step() #applies these gradients to update the model parameters.It typically employs optimization algorithms such as gradient descent or its variants (e.g., Adam, RMSprop) to determine the size and direction of parameter updates.

# # # Evaluation
# # test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
# # model.eval()
# # all_preds = []
# # all_labels = []
# # for batch in test_loader:
# #     input_ids, attention_mask, labels = batch
# #     with torch.no_grad():
# #         outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
# #     logits = outputs.logits
# #     preds = torch.argmax(logits, dim=1).cpu().numpy()
# #     all_preds.extend(preds)
# #     all_labels.extend(labels.numpy())

# # # Compute evaluation metrics
# # from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# # accuracy = accuracy_score(all_labels, all_preds)
# # precision = precision_score(all_labels, all_preds)
# # recall = recall_score(all_labels, all_preds)
# # f1 = f1_score(all_labels, all_preds)

# # print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")


In [74]:
# Assuming train_data is your DataFrame and 'location' is the column of interest
non_string_count = train_data['keyword'].apply(lambda x: not isinstance(x, str)).sum()
print("Number of non-string items in 'location' column:", non_string_count)

Number of non-string items in 'location' column: 0


In [76]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess data
# Assume train_data and test_data are already loaded DataFrames

# Initialize tokenizer and encode text data
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, return_tensors='pt')

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_data['target'].values))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'])

# Define model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training
num_epochs = 2
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# # Evaluation
# valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
# model.eval()
# all_preds = []
# all_labels = []
# for batch in valid_loader:
#     input_ids, attention_mask = batch
#     with torch.no_grad():
#         outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
#     logits = outputs.logits
#     preds = torch.argmax(logits, dim=1).cpu().numpy()
#     all_preds.extend(preds)
#     all_labels.extend(labels.numpy())

# # Compute evaluation metrics
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy = accuracy_score(all_labels, all_preds)
# precision = precision_score(all_labels, all_preds)
# recall = recall_score(all_labels, all_preds)
# f1 = f1_score(all_labels, all_preds)

# print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Evaluation
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
model.eval()
all_preds = []
for batch in test_loader:
    input_ids, attention_mask = batch
    with torch.no_grad():
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    all_preds.extend(preds)

# Prepare predictions for saving in a CSV file
test_ids = test_data['id'].tolist()
predictions = {'id': test_ids, 'target': all_preds}

# Create a DataFrame from predictions
predictions_df = pd.DataFrame(predictions)

# Save predictions to a CSV file
predictions_df.to_csv('test_predictions.csv', index=False)